<a href="https://colab.research.google.com/github/alexandergribenchenko/EDVai_DE/blob/main/EDVai_G_Pyspark_Hive/NB_Pyspark_Hive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Instalación y carga de Pyspark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=a89f0d4e04d8ba987ffda4ba001b09c90dcab9a70e2187774fc4346325080c7c
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

# B. Funciones de pyspark necesarias para el ejercicio

In [36]:
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg, to_date, collect_set, lit

# C. Importar el dataframe raw

### ---> Dataframe de Spark

In [5]:
dfs = spark.read.parquet('yellow_tripdata_2021-01.parquet')

In [6]:
dfs.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|            1.0|          2.1|       1.0|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.

In [7]:
dfs.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



### ---> Vista temporal para consultas SQL

In [12]:
dfs.createOrReplaceTempView('dfs_temp_view')

In [30]:
query = """
SELECT 
    *
FROM 
    dfs_temp_view
"""

In [31]:
spark.sql(query).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|            1.0|          2.1|       1.0|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.

# D. Resolución de las preguntas

## 05. Tabla payments
Insertar en la tabla payments (VendorID, tpep_pickup_datetetime, payment_type,
total_amount) solamente los pagos con tarjeta de crédito.

Sabemos que en `Payment_type` tenemos la equivalencia `1= Credit card`.

### ---> Pyspark version:

In [17]:
dfs.withColumn('tpep_pickup_datetime', to_date('tpep_pickup_datetime', 'yyyy-MM-dd HH:mm:ss')).\
    select('VendorID','tpep_pickup_datetime', 'payment_type', 'total_amount').\
    filter(col('payment_type') == 4).\
    show()

+--------+--------------------+------------+------------+
|VendorID|tpep_pickup_datetime|payment_type|total_amount|
+--------+--------------------+------------+------------+
|       1|          2021-01-01|           4|        28.8|
|       1|          2021-01-01|           4|        37.8|
|       1|          2021-01-01|           4|        16.3|
|       1|          2021-01-01|           4|         8.3|
|       1|          2021-01-01|           4|        10.8|
|       2|          2021-01-01|           4|       -45.3|
|       2|          2021-01-01|           4|        -8.8|
|       2|          2021-01-01|           4|       -15.8|
|       2|          2021-01-01|           4|       -25.8|
|       2|          2021-01-01|           4|       -15.8|
|       1|          2021-01-01|           4|        18.8|
|       2|          2021-01-01|           4|       -19.3|
|       1|          2021-01-01|           4|         6.3|
|       2|          2021-01-01|           4|       -21.8|
|       2|    

### ---> SQL version:

In [22]:
query_05 = """
SELECT 
    VendorID, 
    DATE_FORMAT(tpep_pickup_datetime, 'yyyy-MM-dd') as tpep_pickup_datetime, 
    payment_type, 
    total_amount
FROM 
    dfs_temp_view
WHERE 
    payment_type = 4
"""

In [23]:
spark.sql(query_05).show()

+--------+--------------------+------------+------------+
|VendorID|tpep_pickup_datetime|payment_type|total_amount|
+--------+--------------------+------------+------------+
|       1|          2021-01-01|           4|        28.8|
|       1|          2021-01-01|           4|        37.8|
|       1|          2021-01-01|           4|        16.3|
|       1|          2021-01-01|           4|         8.3|
|       1|          2021-01-01|           4|        10.8|
|       2|          2021-01-01|           4|       -45.3|
|       2|          2021-01-01|           4|        -8.8|
|       2|          2021-01-01|           4|       -15.8|
|       2|          2021-01-01|           4|       -25.8|
|       2|          2021-01-01|           4|       -15.8|
|       1|          2021-01-01|           4|        18.8|
|       2|          2021-01-01|           4|       -19.3|
|       1|          2021-01-01|           4|         6.3|
|       2|          2021-01-01|           4|       -21.8|
|       2|    

## 06. Tabla passengers
Insertar en la tabla passengers (tpep_pickup_datetetime, passenger_count,
total_amount) los registros cuya cantidad de pasajeros sea mayor a 2 y el total del viaje cueste más de 8 dólares.

### ---> Pyspark version:

In [25]:
dfs.withColumn('tpep_pickup_datetime', to_date('tpep_pickup_datetime', 'yyyy-MM-dd HH:mm:ss')).\
    select('tpep_pickup_datetime','passenger_count', 'total_amount').\
    filter((col('passenger_count') > 2) & (col('total_amount') > 8)).\
    show()

+--------------------+---------------+------------+
|tpep_pickup_datetime|passenger_count|total_amount|
+--------------------+---------------+------------+
|          2021-01-01|            3.0|        24.3|
|          2021-01-01|            5.0|       14.16|
|          2021-01-01|            5.0|         8.3|
|          2021-01-01|            3.0|         9.3|
|          2021-01-01|            4.0|        18.3|
|          2021-01-01|            4.0|        13.3|
|          2021-01-01|            3.0|        40.3|
|          2021-01-01|            5.0|        14.8|
|          2021-01-01|            3.0|       18.59|
|          2021-01-01|            3.0|       13.56|
|          2021-01-01|            3.0|        9.96|
|          2021-01-01|            3.0|       66.36|
|          2021-01-01|            3.0|       15.95|
|          2021-01-01|            3.0|        15.8|
|          2021-01-01|            3.0|        13.3|
|          2021-01-01|            3.0|       11.76|
|          2

### ---> SQL version:

In [28]:
query_06 = """
SELECT 
    DATE_FORMAT(tpep_pickup_datetime, 'yyyy-MM-dd') as tpep_pickup_datetime, 
    passenger_count, 
    total_amount
FROM 
    dfs_temp_view
WHERE 
    passenger_count > 2 
    AND total_amount > 8
"""

In [29]:
spark.sql(query_06).show()

+--------------------+---------------+------------+
|tpep_pickup_datetime|passenger_count|total_amount|
+--------------------+---------------+------------+
|          2021-01-01|            3.0|        24.3|
|          2021-01-01|            5.0|       14.16|
|          2021-01-01|            5.0|         8.3|
|          2021-01-01|            3.0|         9.3|
|          2021-01-01|            4.0|        18.3|
|          2021-01-01|            4.0|        13.3|
|          2021-01-01|            3.0|        40.3|
|          2021-01-01|            5.0|        14.8|
|          2021-01-01|            3.0|       18.59|
|          2021-01-01|            3.0|       13.56|
|          2021-01-01|            3.0|        9.96|
|          2021-01-01|            3.0|       66.36|
|          2021-01-01|            3.0|       15.95|
|          2021-01-01|            3.0|        15.8|
|          2021-01-01|            3.0|        13.3|
|          2021-01-01|            3.0|       11.76|
|          2

## 07. Tabla tolls
Insertar en la tabla tolls (tpep_pickup_datetetime, passenger_count, tolls_amount, total_amount) los registros que tengan pago de peajes mayores a 0.1 y cantidad de pasajeros mayores a 1.

### ---> Pyspark version:

In [33]:
dfs.withColumn('tpep_pickup_datetime', to_date('tpep_pickup_datetime', 'yyyy-MM-dd HH:mm:ss')).\
    select('tpep_pickup_datetime','passenger_count', 'tolls_amount', 'total_amount').\
    filter((col('tolls_amount') > 0.1) & (col('passenger_count') > 1)).\
    show()

+--------------------+---------------+------------+------------+
|tpep_pickup_datetime|passenger_count|tolls_amount|total_amount|
+--------------------+---------------+------------+------------+
|          2021-01-01|            2.0|        6.12|       33.92|
|          2021-01-01|            2.0|        6.12|       59.42|
|          2021-01-01|            2.0|        6.12|       35.92|
|          2021-01-01|            6.0|        6.12|        40.1|
|          2021-01-01|            3.0|        6.12|        54.0|
|          2021-01-01|            2.0|         2.8|        34.1|
|          2021-01-01|            4.0|        6.12|       61.42|
|          2021-01-01|            4.0|        6.12|       51.42|
|          2021-01-01|            2.0|       11.75|       12.05|
|          2021-01-01|            6.0|        6.12|       71.42|
|          2021-01-01|            2.0|        6.12|       26.94|
|          2021-01-01|            3.0|        6.12|       27.42|
|          2021-01-01|   

### ---> SQL version:

In [34]:
query_07 = """
SELECT 
    DATE_FORMAT(tpep_pickup_datetime, 'yyyy-MM-dd') as tpep_pickup_datetime, 
    passenger_count, 
    tolls_amount, 
    total_amount
FROM 
    dfs_temp_view
WHERE 
    tolls_amount > 0.1
    AND passenger_count > 1
"""

In [35]:
spark.sql(query_07).show()

+--------------------+---------------+------------+------------+
|tpep_pickup_datetime|passenger_count|tolls_amount|total_amount|
+--------------------+---------------+------------+------------+
|          2021-01-01|            2.0|        6.12|       33.92|
|          2021-01-01|            2.0|        6.12|       59.42|
|          2021-01-01|            2.0|        6.12|       35.92|
|          2021-01-01|            6.0|        6.12|        40.1|
|          2021-01-01|            3.0|        6.12|        54.0|
|          2021-01-01|            2.0|         2.8|        34.1|
|          2021-01-01|            4.0|        6.12|       61.42|
|          2021-01-01|            4.0|        6.12|       51.42|
|          2021-01-01|            2.0|       11.75|       12.05|
|          2021-01-01|            6.0|        6.12|       71.42|
|          2021-01-01|            2.0|        6.12|       26.94|
|          2021-01-01|            3.0|        6.12|       27.42|
|          2021-01-01|   

## 08. Tabla congestion
Insertar en la tabla congestion (tpep_pickup_datetetime, passenger_count,
congestion_surcharge, total_amount) los registros que hayan tenido congestión en los viajes en la fecha 2021-01-18.

### ---> Pyspark version:

In [37]:
dfs.withColumn('tpep_pickup_datetime', to_date('tpep_pickup_datetime', 'yyyy-MM-dd HH:mm:ss')).\
    select('tpep_pickup_datetime','passenger_count', 'congestion_surcharge', 'total_amount').\
    filter((col('tpep_pickup_datetime') == lit('2021-01-18')) & (col('congestion_surcharge') > 0)).\
    show()

+--------------------+---------------+--------------------+------------+
|tpep_pickup_datetime|passenger_count|congestion_surcharge|total_amount|
+--------------------+---------------+--------------------+------------+
|          2021-01-18|            1.0|                 2.5|        10.8|
|          2021-01-18|            1.0|                 2.5|       16.56|
|          2021-01-18|            1.0|                 2.5|       11.16|
|          2021-01-18|            1.0|                 2.5|        11.3|
|          2021-01-18|            1.0|                 2.5|       21.23|
|          2021-01-18|            1.0|                 2.5|       12.96|
|          2021-01-18|            1.0|                 2.5|       13.87|
|          2021-01-18|            1.0|                 2.5|        14.8|
|          2021-01-18|            1.0|                 2.5|       14.14|
|          2021-01-18|            1.0|                 2.5|        20.8|
|          2021-01-18|            1.0|             

### ---> SQL version:

In [40]:
query_08 = """
SELECT 
  to_date(tpep_pickup_datetime, 'yyyy-MM-dd HH:mm:ss') AS tpep_pickup_datetime, 
  passenger_count, 
  congestion_surcharge, 
  total_amount
FROM 
  dfs_temp_view
WHERE 
  to_date(tpep_pickup_datetime, 'yyyy-MM-dd HH:mm:ss') = '2021-01-18'
  AND congestion_surcharge > 0
"""

In [41]:
spark.sql(query_08).show()

+--------------------+---------------+--------------------+------------+
|tpep_pickup_datetime|passenger_count|congestion_surcharge|total_amount|
+--------------------+---------------+--------------------+------------+
|          2021-01-18|            1.0|                 2.5|        10.8|
|          2021-01-18|            1.0|                 2.5|       16.56|
|          2021-01-18|            1.0|                 2.5|       11.16|
|          2021-01-18|            1.0|                 2.5|        11.3|
|          2021-01-18|            1.0|                 2.5|       21.23|
|          2021-01-18|            1.0|                 2.5|       12.96|
|          2021-01-18|            1.0|                 2.5|       13.87|
|          2021-01-18|            1.0|                 2.5|        14.8|
|          2021-01-18|            1.0|                 2.5|       14.14|
|          2021-01-18|            1.0|                 2.5|        20.8|
|          2021-01-18|            1.0|             

## 09. Tabla distance
Insertar en la tabla distance (tpep_pickup_datetetime, passenger_count, trip_distance, total_amount) los registros de la fecha 2020-12-31 que hayan tenido solamente un pasajero (passenger_count = 1) y hayan recorrido más de 15 millas (trip_distance).

### ---> Pyspark version:

In [45]:
dfs.withColumn('tpep_pickup_datetime', to_date('tpep_pickup_datetime', 'yyyy-MM-dd HH:mm:ss')).\
    select('tpep_pickup_datetime','passenger_count', 'trip_distance', 'total_amount').\
    filter((col('tpep_pickup_datetime') == lit('2020-12-31')) & 
           (col('passenger_count') == 1) & 
           (col('trip_distance') >15)).\
    show()

+--------------------+---------------+-------------+------------+
|tpep_pickup_datetime|passenger_count|trip_distance|total_amount|
+--------------------+---------------+-------------+------------+
|          2020-12-31|            1.0|        17.96|        53.3|
+--------------------+---------------+-------------+------------+



### ---> SQL version:

In [48]:
query_09 = """
SELECT 
  to_date(tpep_pickup_datetime, 'yyyy-MM-dd HH:mm:ss') AS tpep_pickup_datetime, 
  passenger_count, 
  trip_distance, 
  total_amount
FROM 
  dfs_temp_view
WHERE 
  to_date(tpep_pickup_datetime, 'yyyy-MM-dd HH:mm:ss') = '2020-12-31'
  AND passenger_count = 1
  AND trip_distance > 15
"""

In [49]:
spark.sql(query_09).show()

+--------------------+---------------+-------------+------------+
|tpep_pickup_datetime|passenger_count|trip_distance|total_amount|
+--------------------+---------------+-------------+------------+
|          2020-12-31|            1.0|        17.96|        53.3|
+--------------------+---------------+-------------+------------+

